In [2]:
import os
from trips_generator.config import *
from trips_generator.weibull_trips import main
import subprocess
from custom_sumo import CustomSumoEnv

scene_src_probabilities = {
    "High Traffic Scenerio": [0.25, 0.25, 0.25, 0.25],
    "Low Traffic Scenerio": [0.25, 0.25, 0.25, 0.25],
    "NS-Traffic Scenerio": [0.45, 0.05, 0.45, 0.05],
    "EW-Traffic Scenerio": [0.05, 0.45, 0.05, 0.45],
}

scene_number_cars = {
    "High Traffic Scenerio": 1000,
    "Low Traffic Scenerio": 150,
    "NS-Traffic Scenerio": 500,
    "EW-Traffic Scenerio": 500,
}


if not os.path.exists("routes_test"):
    os.makedirs("routes_test")
    
    for scene in scene_src_probabilities:
        
        main(
            src_nodes,
            dst_nodes,
            scene_src_probabilities[scene],
            turn_prob,
            emergency_prob,
            shape,
            scene_number_cars[scene],
            total_timesteps,
        )
        subprocess.run(
            [
                "duarouter",
                "-n",
                "./network_details/intersection.net.xml",  # Input network file
                "-t",
                "trips.trips.xml",  # Input trips file
                "-o",
                f"routes_test/intersection_{scene}.rou.xml",  # Output routes file
            ]
        )
    os.remove("trips.trips.xml")
        

In [3]:
route_files = [f"routes_test/intersection_{scene}.rou.xml" for scene in scene_src_probabilities]

In [4]:
for i in range(4):
    env = CustomSumoEnv(
        net_file="network_details/intersection.net.xml",
        route_files=[route_files[i]],
        out_csv_name=f'outputs/Reward2/{i}/RoundRobin',
        min_green=25,
        yellow_time=5,
        delta_time=30,
        use_gui=False,
        num_seconds=5400
    )

    obs, info = env.reset()
    done = False

    action_index = 0  
    while not done:
        action = action_index % 4
        
        next_obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        action_index += 1

    obs, info = env.reset()
    env.close()

 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 6ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 6ms, vehicles TOT 1000 ACT 1 BUF 0)                 tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 996 ACT 1 BUF 0)                  
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 19ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 12ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 5ms, vehicles TOT 150 ACT 0 BUF 0)                  tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 6ms, vehicles TOT 146 ACT 0 BUF 0)                  
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 29ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds
Step #

In [5]:
from stable_baselines3 import DQN

model = DQN.load("agents/dqn_sumo_multi_scene_reward2")

for i in range(4):
    env = CustomSumoEnv(
        net_file="network_details/intersection.net.xml",
        route_files=[route_files[i]],
        out_csv_name=f'outputs/Reward2/{i}/DQN',
        min_green=5,
        yellow_time=5,
        delta_time=10,
        use_gui=False,
        num_seconds=5400
    )
    
    obs, info = env.reset()
    done = False

    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

    obs, info = env.reset()
    env.close()

/home/saksham/miniconda3/envs/sumo-rl/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/home/saksham/miniconda3/envs/sumo-rl/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 16ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 1000 ACT 0 BUF 0)                 tep #4500.00 (1ms ~= 1000.00*RT, ~166000.00UPS, TraCI: 800ms, vehicles TOT 996 ACT 166 BU
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 31ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 10ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 12ms, vehicles TOT 150 ACT 0 BUF 0)                 tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 17ms, vehicles TOT 146 ACT 0 BUF 0)                 
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 27ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 6ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds
Step #

In [6]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

model = PPO.load('agents/ppo_sumo_multi_scene_reward2')

for i in range(4):
    def make_env():
        return CustomSumoEnv(
            net_file="network_details/intersection.net.xml",
            route_files=[route_files[i]],
            out_csv_name=f'outputs/Reward2/{i}/PPO',
            min_green=5,
            yellow_time=5,
            delta_time=10,
            use_gui=False,
            num_seconds=5400
        )
        
    test_env = DummyVecEnv([make_env])

    obs = test_env.reset()
    done = False
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info = test_env.step(action)


/home/saksham/miniconda3/envs/sumo-rl/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/home/saksham/miniconda3/envs/sumo-rl/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 212ms, vehicles TOT 671 ACT 190 BUF 329)            tep #4500.00 (1ms ~= 1000.00*RT, ~199000.00UPS, TraCI: 227ms, vehicles TOT 669 ACT 199 BU
 Retrying in 1 seconds
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 14ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 105ms, vehicles TOT 150 ACT 86 BUF 0)               tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 97ms, vehicles TOT 146 ACT 82 BUF 0)                
 Retrying in 1 seconds
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds
Step #5400.00 (1ms ~= 1000.00*RT, ~200000.00UPS, TraCI: 254ms, vehicles TOT 225 ACT 200 BUtep #4500.00 (0ms ?*RT. ?UPS, TraCI: 257ms, vehicles TOT 225 ACT 201 BUF 271)            
 Retry

In [7]:
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv

model = A2C.load('agents/a2c_sumo_multi_scene_reward2')

for i in range(4):
    def make_env():
        return CustomSumoEnv(
            net_file="network_details/intersection.net.xml",
            route_files=[route_files[i]],
            out_csv_name=f'outputs/Reward2/{i}/A2C',
            min_green=5,
            yellow_time=5,
            delta_time=10,
            use_gui=False,
            num_seconds=5400
        )
        
    test_env = DummyVecEnv([make_env])

    obs = test_env.reset()
    done = False
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info = test_env.step(action)


 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 13ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 73ms, vehicles TOT 1000 ACT 60 BUF 0)               tep #4500.00 (1ms ~= 1000.00*RT, ~126000.00UPS, TraCI: 178ms, vehicles TOT 955 ACT 126 BU
 Retrying in 1 seconds
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 11ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 150 ACT 0 BUF 0)                  tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 12ms, vehicles TOT 146 ACT 0 BUF 0)                 
 Retrying in 1 seconds
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 10ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 16ms, vehicles TOT 500 ACT 0 BUF 0)                 tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 15ms, vehicles TOT 496 ACT 1 BUF 0)                 
 Retry